In [1]:
from config.database import PRODUCTION_HISTORY_DB_NAME, PRODUCTION_HISTORY_COLLECTION_NAME
from database.main import mongo_handler

object_x_query = {"reference": "SADALES_TIKLS_lv_producer_43X-STJ02620895C_43Z-STO01766085R_12624502"}
object_y_query = {"reference": "SADALES_TIKLS_lv_producer_43X-STJ02625769T_43Z-STO01766477A_12628530"}
object_z_query = {"reference": "SADALES_TIKLS_lv_producer_43X-STJ02101362F_43Z-STO01848266I_12587328"}

query_list = [object_x_query, object_y_query, object_z_query]
historical_data_list = []

for query in query_list:
    historical_data_list.append(mongo_handler.retrieve_data(
        PRODUCTION_HISTORY_DB_NAME, PRODUCTION_HISTORY_COLLECTION_NAME, query))


Connected to MongoDB.


In [2]:
from plotly.offline import iplot
import plotly.graph_objects as go

In [9]:
from pandas import DataFrame


def normalize(entry: float, feature_range=(0, 1)):
    min_value, max_value = min(entry), max(entry)
    return [(x - min_value) / (max_value - min_value) * (feature_range[1] - feature_range[0]) + feature_range[0] for x in entry]

def handle_object_vizualization(historical_data: DataFrame, object_name: str):
    x_values = historical_data.index
    y_values = historical_data['production'].tolist()

    if len(x_values) == 0 or len(y_values) == 0:
        print("Error: No valid data found for visualization.")
        return

    print("Max production value for park {}: {} KWh".format(object_name, max(y_values)))
    print("Data is available starting from: {}  {}".format(min(x_values), max(x_values)))

    scatter_traces = [
        go.Scatter(
            x=x_values,
            y=y_values,
            name='Production',
        )
    ]

    fig = go.Figure(data=scatter_traces)

    layout = dict(
        title='Solar park "{}" production in KWh'.format(object_name),
        xaxis=dict(title='Time'),
        yaxis=dict(title='Production')
    )

    fig.update_layout(layout, width=1300, height=600)
    iplot(fig)

for historical_data, object_name in zip(historical_data_list, ['X', 'Y', 'W']):
    handle_object_vizualization(historical_data, object_name)

Max production value for park X: 2962 KWh
Data is available starting from: 2023-06-23 21:00:00  2024-01-15 21:00:00


Max production value for park Y: 1644 KWh
Data is available starting from: 2023-08-30 21:00:00  2024-01-15 21:00:00


Max production value for park W: 227.336 KWh
Data is available starting from: 2023-06-06 21:00:00  2024-01-15 21:00:00
